In [1]:
"""Create parameters.py from a .inp and load it"""
# This cell is facultative, you can use an existing parameters.py

# Directory for .inp file:
path = 'INP file examples\\chem_Otomo2018_without_O.inp'

# import libraries:
import importlib   # for reloading your own files
from termcolor import colored   # for colored error messages

# import inp_data_extractor.py as inp:
try:
    import inp_data_extractor as inp
except ImportError:
    try:
        from Bubble_dynamics_simulation import inp_data_extractor as inp
    except ImportError as error:
        print(colored(f'Error, \'inp_data_extractor.py\' not found', 'red'))
        raise error
    except Exception as error:
        print(colored(f'Error, \'inp_data_extractor.py\' failed to load', 'red'))
        raise error
except Exception as error:
    print(colored(f'Error, \'inp_data_extractor.py\' failed to load', 'red'))
    raise error
importlib.reload(inp)   # reload changes you made

# create parameters.py
inp.extract(path)

# load parameters.py
import parameters as par
importlib.reload(par)
print(par.model)

path=INP file examples\chem_Otomo2018_without_O.inp
Note, lambda value for specie 'H' is not in data.py: 0.0 is used
Note, lambda value for specie 'NH2' is not in data.py: 0.0 is used
Note, lambda value for specie 'NH' is not in data.py: 0.0 is used
Note, lambda value for specie 'N' is not in data.py: 0.0 is used
Note, lambda value for specie 'NNH' is not in data.py: 0.0 is used
Note, lambda value for specie 'N2H4' is not in data.py: 0.0 is used
Note, lambda value for specie 'N2H3' is not in data.py: 0.0 is used
Note, lambda value for specie 'N2H2' is not in data.py: 0.0 is used
Note, lambda value for specie 'H2NN' is not in data.py: 0.0 is used
Warning, third body 'H2O' is not in species in line 64 (' H2/2.5/ H2O/12/ AR/0.0/ ') in reaction 'H2+M=H+H+M'
Warning, third body 'AR' is not in species in line 64 (' H2/2.5/ H2O/12/ AR/0.0/ ') in reaction 'H2+M=H+H+M'
model: chem_Otomo2018_without_O
File 'parameters.py' succesfully created
chem_Otomo2018_without_O


In [2]:
"""Libraries"""

# for plotting:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})

import numpy as np   # matrices, math
import time   # runtime measurement
from multiprocessing import Pool, cpu_count   # multithreading
import importlib   # reload changes you made
import itertools   # assemble all combinations of control parameters
import json   # convert dictionary to string

# import full_bubble_model.py as de:
_already_imported = 'de' in globals()
try:
    import full_bubble_model as de
except ImportError:
    try:
        from  Bubble_dynamics_simulation import full_bubble_model as de
    except ImportError as _error:
        print(colored(f'Error, \'full_bubble_model.py\' not found', 'red'))
        raise _error
    except Exception as _error:
        print(colored(f'Error, \'full_bubble_model.py\' failed to load', 'red'))
        raise _error
except Exception as _error:
    print(colored(f'Error, \'full_bubble_model.py\' failed to load', 'red'))
    raise _error
if _already_imported: importlib.reload(de)   # reload changes you made

model: chem_Otomo2018_without_O
target specie: NH3
excitation: sin_impulse (control parameters: ['p_A', 'freq', 'n'])
enable heat transfer: True	enable evaporation: False	enable reactions: True	enable dissipated energy: True


In [3]:
"""Control parameter ranges and division"""
# a list for each control parameter, containing all the possible values

ranges = dict(
  # Initial conditions:
    # bubble equilibrium radius [m]
    R_E = [1e-6*x for x in np.linspace(5.0, 125.0, 50)], # [um --> m]
    # initial radius / equilibrium radius R_0/R_E [-]
    ratio = [1.0],
  # Ambient parameters:
    # ambient pressure [Pa]
    P_amb = [x * par.atm2Pa for x in [1.0]], # [atm --> Pa]
    # ambient temperature [K]       
    T_inf = [par.absolute_zero + x for x in [20.0]], # [°C --> K]
  # Liquid parameters:
    # water accommodation coefficient [-]
    alfa_M = [0.35],
    # vapour pressure [Pa]
    #P_v = par.P_v, # calculated from T_inf
    # dynamic viscosity [Pa*s]
    #mu_L = par.mu_L, # calculated from T_inf
    # density [kg/m^3]
    rho_L =  [998.20],
    # sound speed [m/s]
    c_L = [par.c_L],
    # surfactant (surface tension modfier) [-]
    surfactant = [1.0],
  # Excitation parameters: (excitation_type = sin_impulse)
    # excitation amplitude [Pa]
    p_A = [-x * par.bar2Pa for x in np.linspace(1.0, 3.0, 50)], # [bar --> Pa]
    # excitation frequency [Hz]
    freq =  [20000.00],
    # excitation duration in period times [-]
    n =  [1.00],
)

for key in de.excitation_args:
    if key not in ranges:
        print(colored(f'Error, {key} not in ranges', 'red'))
# print total combinations:
for key in ranges:
    print(f'{key}: {len(ranges[key])}')
total_combinations = f'total combinations: {np.prod([len(ranges[key]) for key in ranges])}'
print(''.join(['_' for i in range(len(total_combinations))]))
print(total_combinations)

R_E: 50
ratio: 1
P_amb: 1
T_inf: 1
alfa_M: 1
rho_L: 1
c_L: 1
surfactant: 1
p_A: 50
freq: 1
n: 1
________________________
total combinations: 2500


In [4]:
"""Get all combinations"""
# Make a list, with one dictionary for eachy parameter combinations

start = time.time()
cpars = []
ID = 1
for values in itertools.product(*ranges.values()):
    cpar = dict(zip(ranges.keys(), values))
    cpar['ID'] = ID                      # ID of control parameter (not used during calculation)
    cpar['gases'] = [par.index['H2'], par.index['N2']]    # indexes of species in initial bubble (list of species indexes)
    cpar['fractions'] = [0.75, 0.25]            # molar fractions of species in initial bubble (list of fractions for every gas)
    # Calculate pressure/temperature dependent parameters:
    cpar['mu_L'] = de.viscosity(cpar['T_inf'])
    cpar['P_v'] = de.vapour_pressure(cpar['T_inf'])
    cpars.append(cpar)
    ID += 1

print(f'Assemble cpars: {time.time()-start:.2f} s')
start = time.time()

# Create input dictionary for de.simulate(), a list of dictionaries with cpar and other arguments
kwargs_list = [dict(cpar=cpar, t_int=np.array([0.0, 1.0]), LSODA_timeout=30, Radau_timeout=100) for cpar in cpars]
end = time.time()
print(f'Assemble kwargs_list: {time.time()-start:.2f} s')

Assemble cpars: 0.01 s
Assemble kwargs_list: 0.00 s


In [5]:
"""Save settings as txt"""

# create folder for parameter study results:
file = de.Make_dir('test_1atm_20000Hz_2D')

# save all settings (full_bubble_model.py, parameters.py, ranges) as txt:
combined_str = f'''parameters settings:
    model = {par.model}
    species = {par.species}
    number of species = {par.K}
    number of reactions = {par.I}

full_bubble_model settings:
    enable_heat_transfer = {de.enable_heat_transfer}
    enable_evaporation = {de.enable_evaporation} 
    enable_reactions = {de.enable_reactions}
    enable_dissipated_energy = {de.enable_dissipated_energy}
    target_specie = \'{de.target_specie}\' # Specie to calculate energy effiqiency
    excitation_type = \'{de.excitation_type}\' # function to calculate pressure excitation

total_combinations = {np.prod([len(ranges[key]) for key in ranges])}
ranges = {json.dumps(ranges, indent=4)}'''

file.write_string(combined_str, 'brutefroce_parameter_sweep_settings')

In [6]:
"""Parameter study, multithread"""
# Runs each combinations in cpars, and saves the results into CSV files
# use Pool(processes=cpu_count()-1) to limit number of threads being used.
# use pool.imap(...) instead of pool.imap_unordered(...) to preserve order in which cpars was made

max_lines = 1000    # maximum length of a CSV
best_energy_demand = 1e30

start = time.time()
file.new_file()
with Pool(processes=cpu_count(), maxtasksperchild=100) as pool:
    results = pool.imap_unordered(de.simulate, kwargs_list)

    for data in results:
      # save results:
        if file.lines > max_lines:
            file.close()
            file.new_file()
        data = de.dotdict(data)
        file.write_line(data)
      # print stuff:
        if data.energy_demand > 0 and data.energy_demand < best_energy_demand:
            best_energy_demand = data.energy_demand
        excitation_params = ''.join([f'{key}={data[key]: <12}; ' for key in de.excitation_args])
        print(f'index: {data.ID: >8}/{len(cpars)};   error_code: {data.error_code: >4};   steps: {data.steps: <8};   runtime: {data.elapsed_time: 6.2f} [s]   |   '+
              f'R_E={1e6*data.R_E: 6.2f} [um]; ratio={data.ratio: 6.2f} [-]; P_amb={1e-5*data.P_amb: 6.2f} [bar]; T_inf={data.T_inf-273.15: 6.2f} [°C]; '+
              f'alfa_M={data.alfa_M: 6.2f} [-]; P_v={data.P_v: 6.0f} [Pa]; mu_L={data.mu_L: 6.5f} [Pa*s]; rho_L={data.rho_L: 6.1f} [kg/m^3]; c_L={data.c_L: 6.0f} [m/s]; '+
              f'surfactant={100*data.surfactant: 3.0f} [%]   |   {excitation_params}   |   '+
              f'{de.target_specie} production: {data.energy_demand: e} [MJ/kg] (best: {best_energy_demand: .1f} [MJ/kg])'+
              '                                                 ', end='\r')
              
file.close()
end = time.time()
elapsed = end - start
print(f'\n\nDONE')
print(f'total time: {(elapsed / 3600): .0f} hours {((elapsed % 3600) / 60): .0f} mins')
print(f'            {elapsed: .2f} [s]   ({(elapsed / len(    cpars)): .2f} [s/run])')

index:     2488/2500;   error_code:    2;   steps: 12627   ;   runtime:  34.17 [s]   |   R_E= 125.00 [um]; ratio=  1.00 [-]; P_amb=  1.01 [bar]; T_inf= 20.00 [°C]; alfa_M=  0.35 [-]; P_v=  2338 [Pa]; mu_L= 0.00102 [Pa*s]; rho_L= 998.2 [kg/m^3]; c_L=  1483 [m/s]; surfactant= 100 [%]   |   p_A=-251020.4081632653; freq=20000.0     ; n=1.0         ;    |   NH3 production:  4.086804e+06 [MJ/kg] (best:  913.1 [MJ/kg])                                                                             

DONE
total time:  0 hours  11 mins
             668.03 [s]   ( 0.27 [s/run])


In [7]:
# If search is prematurely terminated, close opened file:
file.close()